In [1]:
!pip install efficientnet

import os
os.chdir("../")

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from efficientnet.keras import EfficientNetB5

from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam

from keras.callbacks import ReduceLROnPlateau

import tensorflow as tf
from tensorflow.keras import backend as K


img_list = np.load("../data/processed/img_list.npz", allow_pickle=True)
img_names_list = np.load("../data/processed/image_names_list.npz", allow_pickle=True)
img_list = img_list['arr_0']
img_names_list= img_names_list['arr_0']

df = pd.read_csv("../data/files/train.csv")

df_dict = dict()
for line in range(len(df)):
  df_dict[df["id_code"][line]] = {"diagnosis" : df["diagnosis"][line], "array" : None}

for image_name, img in zip(img_names_list, img_list):
  image_name = image_name.strip(".png")
  df_dict[image_name]["array"] = img


img_list = list()
y_train = list()

for image_name, img_data in df_dict.items():
  label = img_data["diagnosis"]
  image_array = img_data["array"]

  y_train.append(label)
  img_list.append(image_array)


y_train = pd.get_dummies(y_train).values
#y_train = pd.get_dummies(df["diagnosis"]).values
y_train_son = np.ones(y_train.shape, dtype="uint8")
y_train_son[:,4] = y_train[:,4]

for i in range(3,-1,-1):
    y_train_son[:,i] = np.logical_or(y_train[:,i], y_train_son[:,i+1])

img_list = np.array(img_list)
print(img_list.shape)

img_list, x_val, y_train, y_val = train_test_split(img_list,
                                                  y_train_son,
                                                  test_size=0.15,
                                                  random_state=2019,
                                                  shuffle=True)

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
data_generator = datagen.flow(img_list, y_train, batch_size=2, seed=2020)

model = Sequential()
model.add(EfficientNetB5(weights="imagenet", include_top=False, input_shape=(400, 400, 3)))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.00005), metrics=["accuracy"])

lr = ReduceLROnPlateau(monitor="val_loss",  # hatalı olan "val_los" -> "val_loss"
                       patience=3,
                       verbose=1,
                       mode="auto",
                       factor=0.25,
                       min_lr=0.000001)

history = model.fit(data_generator,
                    steps_per_epoch=1000,
                    epochs=10,
                    validation_data=(x_val, y_val),
                    callbacks=[lr])
model.save('../saved_models/diabetic_eye_model_CPU_trained_10_epoch.h5')

(3662, 400, 400, 3)
Epoch 1/10
1000/1000 [==============================] - 2098s 2s/step - loss: 0.3325 - accuracy: 0.9605 - val_loss: 0.1871 - val_accuracy: 0.9982 - lr: 5.0000e-05
Epoch 2/10
1000/1000 [==============================] - 2027s 2s/step - loss: 0.2051 - accuracy: 0.9255 - val_loss: 0.1565 - val_accuracy: 1.0000 - lr: 5.0000e-05
Epoch 3/10
1000/1000 [==============================] - 2063s 2s/step - loss: 0.1812 - accuracy: 0.9245 - val_loss: 0.1463 - val_accuracy: 0.9945 - lr: 5.0000e-05
Epoch 4/10
1000/1000 [==============================] - 2031s 2s/step - loss: 0.1481 - accuracy: 0.8965 - val_loss: 0.1440 - val_accuracy: 1.0000 - lr: 5.0000e-05
Epoch 5/10
1000/1000 [==============================] - 2069s 2s/step - loss: 0.1357 - accuracy: 0.8865 - val_loss: 0.1328 - val_accuracy: 0.9600 - lr: 5.0000e-05
Epoch 6/10
1000/1000 [==============================] - 2044s 2s/step - loss: 0.1279 - accuracy: 0.8790 - val_loss: 0.1224 - val_accuracy: 0.9964 - lr: 5.0000e-05
Ep

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
